<a href="https://colab.research.google.com/github/karenbennis/Xy/blob/ml_model/ml_model_1_3_5_stars_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <br>**Connect to Database**<br><br>

In [1]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

#Interact with SQL
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

# Start Spark Session(Creating spark application with name defined by appName()) ---IMPORTED WITH EVERY COLAB NOTEBOOK
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("database_transformation").config("spark.driver.memory","5g").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()


--2020-07-26 15:58:27--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.42MB/s    in 0.6s    

2020-07-26 15:58:29 (1.42 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [ ]:
# gcloud login and check the DB
!gcloud auth login
!gcloud config set project 'xy-yelp'
!gcloud sql instances describe 'xy-yelp'

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&code_challenge=TGHQzzth5e5AMSwqAw6CpiH3wec9voSu-hui-MK-he0&code_challenge_method=S256&access_type=offline&response_type=code&prompt=select_account


Enter verification code: 4/2QHe7YUZLypxZoQQ6PhcY2igPX1F0aY8znLdi8Pv6YIxOPyVrLPM_vU

You are now logged in as [helenly25@gmail.com].
Your current project is [xy-yelp].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID
Updated property [core/project].
backendType: SECOND_GEN
connectionName: xy-yelp:northamerica-northeast1:xy-yelp
databaseV

In [ ]:
# download and initialize the psql proxy
!wget https://dl.google.com/cloudsql/cloud_sql_proxy.linux.amd64 -O cloud_sql_proxy
!chmod +x cloud_sql_proxy
# "connectionName" is from the previous block
!nohup ./cloud_sql_proxy -instances="xy-yelp:northamerica-northeast1:xy-yelp"=tcp:5432 &
!sleep 30s

cloud_sql_proxy: Text file busy
nohup: appending output to 'nohup.out'


In [ ]:
db_password = 'kjhbyelpdb'

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://127.0.0.1:5432/xy_yelp_db"
config = {"user":"postgres", 
          "password": db_password, 
          "driver":"org.postgresql.Driver"}

## **Extract tables**

In [2]:
# convert to pandas
import pandas as pd
pandas_df = pd.read_csv("https://raw.githubusercontent.com/karenbennis/Xy/storyboard/uniform_yelp.csv")

# Set index
# pandas_df = pandas_df.set_index('review_id')

pandas_df.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,f5pGCpvkpRpJixZ0zA3hCg,8p2nss7UoZmIVZTr1IjR3w,caWUE0ItqsG51OaBVlr4Eg,2,2016-11-13,We went here three more times for lunch and tw...,0,0,0
1,8lPKsNFBiLmVL5nbsUXaZw,O3pSxv1SyHpY4qi4Q16KzA,dc3uoAmNo5STqKV6mlD_aA,1,2017-05-30,My husband and I went to the Drake for lunch t...,2,1,1
2,CANhCLzOoZ0mkL3mpnUSNg,ffC9zmbY4pBOS9ByrWoXxQ,sR9hPrIaG-J-GLcl4yaiLw,1,2017-11-19,"Very Problematic\nI'm gay, I'm not ashamed to ...",0,0,0
3,bZ02moAXlosgWPM3pXSHWw,zE49S2Em3l7vgIlvFzZFOw,NF6di6YcQxN0rDAleE7SyQ,3,2014-12-30,Today was the first time I sat down a table. I...,0,0,0
4,TmIla5Eh5SSLJ_bKgH4Syg,xde2rO3XVt0Do8kLRIt2Dw,Wc9UpJhOcdSj7olZkz7SJA,2,2013-02-19,I ordered chicken tacos with no cheese to go. ...,4,1,0


## **Transformation**

In [7]:
pandas_df = pandas_df.loc[(pandas_df['stars'] != 2) & (pandas_df['stars'] != 4)]
pandas_df.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
1,8lPKsNFBiLmVL5nbsUXaZw,O3pSxv1SyHpY4qi4Q16KzA,dc3uoAmNo5STqKV6mlD_aA,1,2017-05-30,My husband and I went to the Drake for lunch t...,2,1,1
2,CANhCLzOoZ0mkL3mpnUSNg,ffC9zmbY4pBOS9ByrWoXxQ,sR9hPrIaG-J-GLcl4yaiLw,1,2017-11-19,"Very Problematic\nI'm gay, I'm not ashamed to ...",0,0,0
3,bZ02moAXlosgWPM3pXSHWw,zE49S2Em3l7vgIlvFzZFOw,NF6di6YcQxN0rDAleE7SyQ,3,2014-12-30,Today was the first time I sat down a table. I...,0,0,0
6,zNXXGTnGOo9pKCN5JyKaPw,YT4FKLW7YiHDdlXZpqjMHg,y3bMutme81x4PUhb0zaC_A,3,2013-05-27,Went to lunch Sunday of Memorial Day weekend. ...,0,1,0
7,Ez-8sHPrQdnyv5YZW26V4w,9Xmw_WcUCShPD0qGO1UD7w,77N3luh2YzfJFJGFIfqqbw,5,2017-04-04,Beer baby!! Always love the selection Duke's o...,0,0,0


In [8]:
# Import dependencies for nltk
# https://towardsdatascience.com/natural-language-processing-nlp-for-machine-learning-d44498845d5b
import nltk

In [9]:
# Import string and punctuations
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [10]:
# Function to remove Punctuation
def remove_punct(text):

  # Discard all punctuations
  text_nopunct = ''.join([char for char in text if char not in string.punctuation])
  return text_nopunct

pandas_df['body_text_clean'] = pandas_df['text'].apply(lambda x: remove_punct(x))

pandas_df.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,body_text_clean
1,8lPKsNFBiLmVL5nbsUXaZw,O3pSxv1SyHpY4qi4Q16KzA,dc3uoAmNo5STqKV6mlD_aA,1,2017-05-30,My husband and I went to the Drake for lunch t...,2,1,1,My husband and I went to the Drake for lunch t...
2,CANhCLzOoZ0mkL3mpnUSNg,ffC9zmbY4pBOS9ByrWoXxQ,sR9hPrIaG-J-GLcl4yaiLw,1,2017-11-19,"Very Problematic\nI'm gay, I'm not ashamed to ...",0,0,0,Very Problematic\nIm gay Im not ashamed to say...
3,bZ02moAXlosgWPM3pXSHWw,zE49S2Em3l7vgIlvFzZFOw,NF6di6YcQxN0rDAleE7SyQ,3,2014-12-30,Today was the first time I sat down a table. I...,0,0,0,Today was the first time I sat down a table I ...
6,zNXXGTnGOo9pKCN5JyKaPw,YT4FKLW7YiHDdlXZpqjMHg,y3bMutme81x4PUhb0zaC_A,3,2013-05-27,Went to lunch Sunday of Memorial Day weekend. ...,0,1,0,Went to lunch Sunday of Memorial Day weekend T...
7,Ez-8sHPrQdnyv5YZW26V4w,9Xmw_WcUCShPD0qGO1UD7w,77N3luh2YzfJFJGFIfqqbw,5,2017-04-04,Beer baby!! Always love the selection Duke's o...,0,0,0,Beer baby Always love the selection Dukes offe...


In [11]:
# Import re
import re

# Function to Tokenize words
def tokenize(text):

  # W+ means that either a word character (A-Za-z0-9) or a dash (-) can go there
  tokens = re.split('\W+', text)
  return tokens

# Convert to lowercase as Python is case-sensitive
pandas_df['body_text_tokenized'] = pandas_df['body_text_clean'].apply(lambda x: tokenize(x.lower()))

pandas_df.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,body_text_clean,body_text_tokenized
1,8lPKsNFBiLmVL5nbsUXaZw,O3pSxv1SyHpY4qi4Q16KzA,dc3uoAmNo5STqKV6mlD_aA,1,2017-05-30,My husband and I went to the Drake for lunch t...,2,1,1,My husband and I went to the Drake for lunch t...,"[my, husband, and, i, went, to, the, drake, fo..."
2,CANhCLzOoZ0mkL3mpnUSNg,ffC9zmbY4pBOS9ByrWoXxQ,sR9hPrIaG-J-GLcl4yaiLw,1,2017-11-19,"Very Problematic\nI'm gay, I'm not ashamed to ...",0,0,0,Very Problematic\nIm gay Im not ashamed to say...,"[very, problematic, im, gay, im, not, ashamed,..."
3,bZ02moAXlosgWPM3pXSHWw,zE49S2Em3l7vgIlvFzZFOw,NF6di6YcQxN0rDAleE7SyQ,3,2014-12-30,Today was the first time I sat down a table. I...,0,0,0,Today was the first time I sat down a table I ...,"[today, was, the, first, time, i, sat, down, a..."
6,zNXXGTnGOo9pKCN5JyKaPw,YT4FKLW7YiHDdlXZpqjMHg,y3bMutme81x4PUhb0zaC_A,3,2013-05-27,Went to lunch Sunday of Memorial Day weekend. ...,0,1,0,Went to lunch Sunday of Memorial Day weekend T...,"[went, to, lunch, sunday, of, memorial, day, w..."
7,Ez-8sHPrQdnyv5YZW26V4w,9Xmw_WcUCShPD0qGO1UD7w,77N3luh2YzfJFJGFIfqqbw,5,2017-04-04,Beer baby!! Always love the selection Duke's o...,0,0,0,Beer baby Always love the selection Dukes offe...,"[beer, baby, always, love, the, selection, duk..."


In [12]:
# Remove all English stopwords
import nltk
nltk.download('stopwords')
stopword = nltk.corpus.stopwords.words('english') 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
# Function to remove stopwords
def remove_stopwords(tokenized_list):

  # Remove all stopwords
  text = [word for word in tokenized_list if word not in stopword]
  return text

pandas_df['body_text_nostop'] = pandas_df['body_text_tokenized'].apply(lambda x: remove_stopwords(x))

pandas_df.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,body_text_clean,body_text_tokenized,body_text_nostop
1,8lPKsNFBiLmVL5nbsUXaZw,O3pSxv1SyHpY4qi4Q16KzA,dc3uoAmNo5STqKV6mlD_aA,1,2017-05-30,My husband and I went to the Drake for lunch t...,2,1,1,My husband and I went to the Drake for lunch t...,"[my, husband, and, i, went, to, the, drake, fo...","[husband, went, drake, lunch, today, sat, pati..."
2,CANhCLzOoZ0mkL3mpnUSNg,ffC9zmbY4pBOS9ByrWoXxQ,sR9hPrIaG-J-GLcl4yaiLw,1,2017-11-19,"Very Problematic\nI'm gay, I'm not ashamed to ...",0,0,0,Very Problematic\nIm gay Im not ashamed to say...,"[very, problematic, im, gay, im, not, ashamed,...","[problematic, im, gay, im, ashamed, say, felt,..."
3,bZ02moAXlosgWPM3pXSHWw,zE49S2Em3l7vgIlvFzZFOw,NF6di6YcQxN0rDAleE7SyQ,3,2014-12-30,Today was the first time I sat down a table. I...,0,0,0,Today was the first time I sat down a table I ...,"[today, was, the, first, time, i, sat, down, a...","[today, first, time, sat, table, like, wait, w..."
6,zNXXGTnGOo9pKCN5JyKaPw,YT4FKLW7YiHDdlXZpqjMHg,y3bMutme81x4PUhb0zaC_A,3,2013-05-27,Went to lunch Sunday of Memorial Day weekend. ...,0,1,0,Went to lunch Sunday of Memorial Day weekend T...,"[went, to, lunch, sunday, of, memorial, day, w...","[went, lunch, sunday, memorial, day, weekend, ..."
7,Ez-8sHPrQdnyv5YZW26V4w,9Xmw_WcUCShPD0qGO1UD7w,77N3luh2YzfJFJGFIfqqbw,5,2017-04-04,Beer baby!! Always love the selection Duke's o...,0,0,0,Beer baby Always love the selection Dukes offe...,"[beer, baby, always, love, the, selection, duk...","[beer, baby, always, love, selection, dukes, o..."


In [14]:
# Import PorterStemmer
from nltk.stem import PorterStemmer

# Create an instance for stemmer
ps = nltk.PorterStemmer()

# Function for stemming
def stemming(tokenized_text):

  text = [ps.stem(word) for word in tokenized_text]
  return text

pandas_df['body_text_stemmed'] = pandas_df['body_text_nostop'].apply(lambda x: stemming(x))

pandas_df.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,body_text_clean,body_text_tokenized,body_text_nostop,body_text_stemmed
1,8lPKsNFBiLmVL5nbsUXaZw,O3pSxv1SyHpY4qi4Q16KzA,dc3uoAmNo5STqKV6mlD_aA,1,2017-05-30,My husband and I went to the Drake for lunch t...,2,1,1,My husband and I went to the Drake for lunch t...,"[my, husband, and, i, went, to, the, drake, fo...","[husband, went, drake, lunch, today, sat, pati...","[husband, went, drake, lunch, today, sat, pati..."
2,CANhCLzOoZ0mkL3mpnUSNg,ffC9zmbY4pBOS9ByrWoXxQ,sR9hPrIaG-J-GLcl4yaiLw,1,2017-11-19,"Very Problematic\nI'm gay, I'm not ashamed to ...",0,0,0,Very Problematic\nIm gay Im not ashamed to say...,"[very, problematic, im, gay, im, not, ashamed,...","[problematic, im, gay, im, ashamed, say, felt,...","[problemat, im, gay, im, asham, say, felt, ash..."
3,bZ02moAXlosgWPM3pXSHWw,zE49S2Em3l7vgIlvFzZFOw,NF6di6YcQxN0rDAleE7SyQ,3,2014-12-30,Today was the first time I sat down a table. I...,0,0,0,Today was the first time I sat down a table I ...,"[today, was, the, first, time, i, sat, down, a...","[today, first, time, sat, table, like, wait, w...","[today, first, time, sat, tabl, like, wait, we..."
6,zNXXGTnGOo9pKCN5JyKaPw,YT4FKLW7YiHDdlXZpqjMHg,y3bMutme81x4PUhb0zaC_A,3,2013-05-27,Went to lunch Sunday of Memorial Day weekend. ...,0,1,0,Went to lunch Sunday of Memorial Day weekend T...,"[went, to, lunch, sunday, of, memorial, day, w...","[went, lunch, sunday, memorial, day, weekend, ...","[went, lunch, sunday, memori, day, weekend, mu..."
7,Ez-8sHPrQdnyv5YZW26V4w,9Xmw_WcUCShPD0qGO1UD7w,77N3luh2YzfJFJGFIfqqbw,5,2017-04-04,Beer baby!! Always love the selection Duke's o...,0,0,0,Beer baby Always love the selection Dukes offe...,"[beer, baby, always, love, the, selection, duk...","[beer, baby, always, love, selection, dukes, o...","[beer, babi, alway, love, select, duke, offer,..."


In [15]:
# import WordNetLemmatizer 
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 

# Create an instance
wn = nltk.WordNetLemmatizer()

# Function for lemmatization
def lemmatizing(tokenized_text):

  text = [wn.lemmatize(word) for word in tokenized_text]
  return text

pandas_df['body_text_lemmatized'] = pandas_df['body_text_nostop'].apply(lambda x: lemmatizing(x))

pandas_df.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,review_id,user_id,business_id,stars,date,text,useful,funny,cool,body_text_clean,body_text_tokenized,body_text_nostop,body_text_stemmed,body_text_lemmatized
1,8lPKsNFBiLmVL5nbsUXaZw,O3pSxv1SyHpY4qi4Q16KzA,dc3uoAmNo5STqKV6mlD_aA,1,2017-05-30,My husband and I went to the Drake for lunch t...,2,1,1,My husband and I went to the Drake for lunch t...,"[my, husband, and, i, went, to, the, drake, fo...","[husband, went, drake, lunch, today, sat, pati...","[husband, went, drake, lunch, today, sat, pati...","[husband, went, drake, lunch, today, sat, pati..."
2,CANhCLzOoZ0mkL3mpnUSNg,ffC9zmbY4pBOS9ByrWoXxQ,sR9hPrIaG-J-GLcl4yaiLw,1,2017-11-19,"Very Problematic\nI'm gay, I'm not ashamed to ...",0,0,0,Very Problematic\nIm gay Im not ashamed to say...,"[very, problematic, im, gay, im, not, ashamed,...","[problematic, im, gay, im, ashamed, say, felt,...","[problemat, im, gay, im, asham, say, felt, ash...","[problematic, im, gay, im, ashamed, say, felt,..."
3,bZ02moAXlosgWPM3pXSHWw,zE49S2Em3l7vgIlvFzZFOw,NF6di6YcQxN0rDAleE7SyQ,3,2014-12-30,Today was the first time I sat down a table. I...,0,0,0,Today was the first time I sat down a table I ...,"[today, was, the, first, time, i, sat, down, a...","[today, first, time, sat, table, like, wait, w...","[today, first, time, sat, tabl, like, wait, we...","[today, first, time, sat, table, like, wait, w..."
6,zNXXGTnGOo9pKCN5JyKaPw,YT4FKLW7YiHDdlXZpqjMHg,y3bMutme81x4PUhb0zaC_A,3,2013-05-27,Went to lunch Sunday of Memorial Day weekend. ...,0,1,0,Went to lunch Sunday of Memorial Day weekend T...,"[went, to, lunch, sunday, of, memorial, day, w...","[went, lunch, sunday, memorial, day, weekend, ...","[went, lunch, sunday, memori, day, weekend, mu...","[went, lunch, sunday, memorial, day, weekend, ..."
7,Ez-8sHPrQdnyv5YZW26V4w,9Xmw_WcUCShPD0qGO1UD7w,77N3luh2YzfJFJGFIfqqbw,5,2017-04-04,Beer baby!! Always love the selection Duke's o...,0,0,0,Beer baby Always love the selection Dukes offe...,"[beer, baby, always, love, the, selection, duk...","[beer, baby, always, love, selection, dukes, o...","[beer, babi, alway, love, select, duke, offer,...","[beer, baby, always, love, selection, duke, of..."


In [16]:
# Add a length column to DataFrame
pandas_df['length'] = pandas_df['text'].apply(len)
pandas_df.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,body_text_clean,body_text_tokenized,body_text_nostop,body_text_stemmed,body_text_lemmatized,length
1,8lPKsNFBiLmVL5nbsUXaZw,O3pSxv1SyHpY4qi4Q16KzA,dc3uoAmNo5STqKV6mlD_aA,1,2017-05-30,My husband and I went to the Drake for lunch t...,2,1,1,My husband and I went to the Drake for lunch t...,"[my, husband, and, i, went, to, the, drake, fo...","[husband, went, drake, lunch, today, sat, pati...","[husband, went, drake, lunch, today, sat, pati...","[husband, went, drake, lunch, today, sat, pati...",407
2,CANhCLzOoZ0mkL3mpnUSNg,ffC9zmbY4pBOS9ByrWoXxQ,sR9hPrIaG-J-GLcl4yaiLw,1,2017-11-19,"Very Problematic\nI'm gay, I'm not ashamed to ...",0,0,0,Very Problematic\nIm gay Im not ashamed to say...,"[very, problematic, im, gay, im, not, ashamed,...","[problematic, im, gay, im, ashamed, say, felt,...","[problemat, im, gay, im, asham, say, felt, ash...","[problematic, im, gay, im, ashamed, say, felt,...",483
3,bZ02moAXlosgWPM3pXSHWw,zE49S2Em3l7vgIlvFzZFOw,NF6di6YcQxN0rDAleE7SyQ,3,2014-12-30,Today was the first time I sat down a table. I...,0,0,0,Today was the first time I sat down a table I ...,"[today, was, the, first, time, i, sat, down, a...","[today, first, time, sat, table, like, wait, w...","[today, first, time, sat, tabl, like, wait, we...","[today, first, time, sat, table, like, wait, w...",1682
6,zNXXGTnGOo9pKCN5JyKaPw,YT4FKLW7YiHDdlXZpqjMHg,y3bMutme81x4PUhb0zaC_A,3,2013-05-27,Went to lunch Sunday of Memorial Day weekend. ...,0,1,0,Went to lunch Sunday of Memorial Day weekend T...,"[went, to, lunch, sunday, of, memorial, day, w...","[went, lunch, sunday, memorial, day, weekend, ...","[went, lunch, sunday, memori, day, weekend, mu...","[went, lunch, sunday, memorial, day, weekend, ...",296
7,Ez-8sHPrQdnyv5YZW26V4w,9Xmw_WcUCShPD0qGO1UD7w,77N3luh2YzfJFJGFIfqqbw,5,2017-04-04,Beer baby!! Always love the selection Duke's o...,0,0,0,Beer baby Always love the selection Dukes offe...,"[beer, baby, always, love, the, selection, duk...","[beer, baby, always, love, selection, dukes, o...","[beer, babi, alway, love, select, duke, offer,...","[beer, baby, always, love, selection, duke, of...",579


## <br></br>**Pipeline**<br></br>

In [17]:
# Make a copy of data
pandas_df_copy = pandas_df.copy()

# Select columns for new DataFrame
pandas_df_copy = pandas_df_copy[['review_id', 'text', 'stars', 'cool', 'useful', 'funny', 'date', 'business_id', 'user_id', 'length', 'body_text_nostop', 'body_text_stemmed', 'body_text_lemmatized']]

# Convert pandas_df to sparks df
spark_df = spark.createDataFrame(pandas_df_copy)
spark_df.show(5)

+--------------------+--------------------+-----+----+------+-----+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+
|           review_id|                text|stars|cool|useful|funny|      date|         business_id|             user_id|length|    body_text_nostop|   body_text_stemmed|body_text_lemmatized|
+--------------------+--------------------+-----+----+------+-----+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+
|8lPKsNFBiLmVL5nbs...|My husband and I ...|    1|   1|     2|    1|2017-05-30|dc3uoAmNo5STqKV6m...|O3pSxv1SyHpY4qi4Q...|   407|[husband, went, d...|[husband, went, d...|[husband, went, d...|
|CANhCLzOoZ0mkL3mp...|Very Problematic
...|    1|   0|     0|    0|2017-11-19|sR9hPrIaG-J-GLcl4...|ffC9zmbY4pBOS9Byr...|   483|[problematic, im,...|[problemat, im, g...|[problematic, im,...|
|bZ02moAXlosgWPM3p...|Today was the fir...|  

In [18]:
# Import functions
from pyspark.ml.feature import HashingTF, IDF, StringIndexer

In [19]:
# Make stars values a list
from pyspark.sql.functions import col, split
spark_df = spark_df.withColumn("star_array", split(col("stars"), " "))
spark_df.show()

+--------------------+--------------------+-----+----+------+-----+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+----------+
|           review_id|                text|stars|cool|useful|funny|      date|         business_id|             user_id|length|    body_text_nostop|   body_text_stemmed|body_text_lemmatized|star_array|
+--------------------+--------------------+-----+----+------+-----+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+----------+
|8lPKsNFBiLmVL5nbs...|My husband and I ...|    1|   1|     2|    1|2017-05-30|dc3uoAmNo5STqKV6m...|O3pSxv1SyHpY4qi4Q...|   407|[husband, went, d...|[husband, went, d...|[husband, went, d...|       [1]|
|CANhCLzOoZ0mkL3mp...|Very Problematic
...|    1|   0|     0|    0|2017-11-19|sR9hPrIaG-J-GLcl4...|ffC9zmbY4pBOS9Byr...|   483|[problematic, im,...|[problemat, im, g...|[problematic, im,...|  

In [20]:
# Initialize a CoutVectorizer
from pyspark.ml.feature import CountVectorizer
star_vectorizer = CountVectorizer(inputCol="star_array", outputCol="stars_one_hot", vocabSize=5, minDF=1.0)

In [21]:
# Create a vector model
star_vector_model = star_vectorizer.fit(spark_df)

In [22]:
# One hot encoded column
df_ohe = star_vector_model.transform(spark_df)
df_ohe.show(3)

+--------------------+--------------------+-----+----+------+-----+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+----------+-------------+
|           review_id|                text|stars|cool|useful|funny|      date|         business_id|             user_id|length|    body_text_nostop|   body_text_stemmed|body_text_lemmatized|star_array|stars_one_hot|
+--------------------+--------------------+-----+----+------+-----+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+----------+-------------+
|8lPKsNFBiLmVL5nbs...|My husband and I ...|    1|   1|     2|    1|2017-05-30|dc3uoAmNo5STqKV6m...|O3pSxv1SyHpY4qi4Q...|   407|[husband, went, d...|[husband, went, d...|[husband, went, d...|       [1]|(3,[2],[1.0])|
|CANhCLzOoZ0mkL3mp...|Very Problematic
...|    1|   0|     0|    0|2017-11-19|sR9hPrIaG-J-GLcl4...|ffC9zmbY4pBOS9Byr...|   483|[problema

In [23]:
# Create all the features to the data set
star_rating = StringIndexer(inputCol='stars',outputCol='label')
hashingTF = HashingTF(inputCol="body_text_stemmed", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [24]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector
# Create feature vector 
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [25]:
# Create and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[star_rating, hashingTF, idf, clean_up])

In [26]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(df_ohe)
cleaned = cleaner.transform(df_ohe)

In [27]:
cleaned.show(5)

+--------------------+--------------------+-----+----+------+-----+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+----------+-------------+-----+--------------------+--------------------+--------------------+
|           review_id|                text|stars|cool|useful|funny|      date|         business_id|             user_id|length|    body_text_nostop|   body_text_stemmed|body_text_lemmatized|star_array|stars_one_hot|label|          hash_token|           idf_token|            features|
+--------------------+--------------------+-----+----+------+-----+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+----------+-------------+-----+--------------------+--------------------+--------------------+
|8lPKsNFBiLmVL5nbs...|My husband and I ...|    1|   1|     2|    1|2017-05-30|dc3uoAmNo5STqKV6m...|O3pSxv1SyHpY4qi4Q...|   407|[husband, went, d.

# **Machine Learning Models**

In [28]:
#Drop intermediate columns
x=cleaned.select('features', 'label')
x.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(262145,[6872,156...|  2.0|
|(262145,[3067,690...|  2.0|
|(262145,[1353,232...|  0.0|
|(262145,[1353,139...|  0.0|
|(262145,[22567,29...|  1.0|
+--------------------+-----+
only showing top 5 rows



In [29]:
# Check dtypes
x.dtypes

[('features', 'vector'), ('label', 'double')]

In [30]:
# Import col
from pyspark.sql.functions import col

# Change column DataType for stars
x = x.withColumn('label', col('label').cast('int'))

x.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(262145,[6872,156...|    2|
|(262145,[3067,690...|    2|
|(262145,[1353,232...|    0|
|(262145,[1353,139...|    0|
|(262145,[22567,29...|    1|
+--------------------+-----+
only showing top 5 rows



**Naive Bayes**

In [31]:
# Break data down into a training set and a testing set
training, testing = x.randomSplit([0.8, 0.2], 21)

In [32]:
from pyspark.ml.classification import NaiveBayes
# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [33]:
# Use the Class Evaluator for a cleaned description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(predictor.transform(testing))
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.504272


**Logistic Regression**

In [34]:
# Break data down into a training set and a testing set
training, testing = x.randomSplit([0.8, 0.2], 21)

In [35]:
from pyspark.ml.classification import LogisticRegression
# Create a Naive Bayes model and fit training data
lg = LogisticRegression()
predictor = lg.fit(training)

In [36]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(predictor.transform(testing))
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.740741


**Multilayer Percepron**

In [37]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [38]:
# Split the data into train and test
splits = x.randomSplit([0.8, 0.2], 1234)
train = splits[0]
test = splits[1]

In [39]:
train.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(262145,[97,13957...|    1|
|(262145,[98,6646,...|    2|
|(262145,[170,427,...|    2|
|(262145,[170,976,...|    0|
|(262145,[234,1395...|    2|
|(262145,[234,1729...|    1|
|(262145,[353,783,...|    0|
|(262145,[353,976,...|    2|
|(262145,[353,1353...|    0|
|(262145,[353,1707...|    1|
|(262145,[353,3028...|    2|
|(262145,[353,1034...|    1|
|(262145,[353,2234...|    1|
|(262145,[382,2437...|    2|
|(262145,[427,1560...|    0|
|(262145,[427,2325...|    0|
|(262145,[427,3811...|    0|
|(262145,[535,2089...|    2|
|(262145,[584,1353...|    2|
|(262145,[604,6981...|    0|
+--------------------+-----+
only showing top 20 rows



In [40]:
# specify layers for the neural network:
layers = [62000, 256, 3]

In [41]:
# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=10, layers=layers, blockSize=128, seed=1234)

In [42]:
# train the model
model = trainer.fit(train)

Py4JJavaError: ignored